# Explore here

In [13]:
import pandas as pd
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [14]:
df_movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
df_credits = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")

In [15]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [16]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [17]:
df_movies.sample(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
596,70000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,8427,"[{""id"": 810, ""name"": ""budapest""}, {""id"": 1930,...",en,I Spy,"When the Switchblade, the most sophisticated p...",13.267631,"[{""name"": ""Columbia Pictures Corporation"", ""id...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-10-31,33561137,97.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Attitude meets espionage.,I Spy,5.2,269
1881,25000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 80, ""name...",NaN,278,"[{""id"": 378, ""name"": ""prison""}, {""id"": 417, ""n...",en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,136.747729,"[{""name"": ""Castle Rock Entertainment"", ""id"": 97}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1994-09-23,28341469,142.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,8.5,8205
2412,17000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",http://www.smokinaces.net/,7516,"[{""id"": 376, ""name"": ""neo-nazi""}, {""id"": 1419,...",en,Smokin' Aces,When a Las Vegas performer-turned-snitch named...,23.491962,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""FR"", ""name"": ""France""}, {""iso...",2006-12-09,57103895,109.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Nobody gets away clean.,Smokin' Aces,6.4,534
2840,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",http://www.topfivemovie.com/,284296,"[{""id"": 3485, ""name"": ""comedian""}, {""id"": 4434...",en,Top Five,A comedian tries to make it as a serious actor...,15.896312,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2014-12-12,25434291,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Top Five,6.3,210
3990,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,55306,"[{""id"": 494, ""name"": ""father son relationship""...",en,Ulee's Gold,An elderly beekeeper and Vietnam vet must resc...,0.933920,"[{""name"": ""Clinica Estetico"", ""id"": 1274}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1997-06-13,0,112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Ulee's Gold,6.5,19
372,92000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,1535,"[{""id"": 470, ""name"": ""spy""}, {""id"": 478, ""name...",en,Spy Game,Veteran spy Nathan Muir is on the verge of ret...,27.166757,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2001-11-18,143049560,126.0,"[{""iso_639_1"": ""cn"", ""name"": ""\u5e7f\u5dde\u8b...",Released,It's not how you play the game. It's how the g...,Spy Game,6.8,579
450,80000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""...",NaN,9574,"[{""id"": 1334, ""name"": ""wedding vows""}, {""id"": ...",en,Flubber,"Professor Phillip Brainard, an absent minded p...",11.503960,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1997-11-26,177977226,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Catch it if you can!,Flubber,5.3,695
2236,0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,82684,"[{""id"": 1669, ""name"": ""wave""}, {""id"": 5349, ""n...",en,Chasing Mavericks,Surfer Jay Moriarity sets out to ride the Nort...,13.621201,"[{""name"": ""Fox 2000 Pictures"", ""id"": 711}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-10-25,0,117.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Legends Start Somewhere,Chasing Mavericks,7.0,216
2002,23000000,"[{""id"": 28, ""na

In [18]:
df_credits.sample(10)

,movie_id,title,cast,crew
4142,119458,Supercapitalist,"[{""cast_id"": 11, ""character"": ""Mark Patterson""...","[{""credit_id"": ""52fe4c02c3a36847f821d22d"", ""de..."
1528,302156,Criminal,"[{""cast_id"": 3, ""character"": ""Jericho Stewart""...","[{""credit_id"": ""576528eb92514167de001225"", ""de..."
479,77951,Walking With Dinosaurs,"[{""cast_id"": 10, ""character"": ""Ricky (voice)"",...","[{""credit_id"": ""52fe4985c3a368484e12fc0f"", ""de..."
3218,272693,The DUFF,"[{""cast_id"": 2, ""character"": ""Bianca Piper"", ""...","[{""credit_id"": ""549c98e1c3a3680b27003697"", ""de..."
3744,165864,Hansel and Gretel Get Baked,"[{""cast_id"": 10, ""character"": ""Gretel"", ""credi...","[{""credit_id"": ""52fe4c93c3a36847f8234cff"", ""de..."
1341,16911,The Inhabited Island,"[{""cast_id"": 1, ""character"": ""Maxim Kammerer"",...","[{""credit_id"": ""54e46c62c3a368454b00a240"", ""de..."
3429,1591,Nowhere in Africa,"[{""cast_id"": 17, ""character"": ""Jettel Redlich""...","[{""credit_id"": ""52fe4302c3a36847f803392f"", ""de..."
2045,1599,I Heart Huckabees,"[{""cast_id"": 1, ""character"": ""Albert Markovski...","[{""credit_id"": ""52fe4303c3a36847f8033e31"", ""de..."
1203,1586,Secret Window,"[{""cast_id"": 1, ""character"": ""Mort Rainey"", ""c...","[{""credit_id"": ""52fe4301c3a36847f8033655"", ""de..."
4114,10656,Subway,"[{""cast_id"": 1, ""character"": ""H\u00e9l\u00e9na...","[{""credit_id"": ""52fe439d9251416c75017483"", ""de..."


In [19]:

# Create a SQLite database connection and load DataFrames into tables
conn = sqlite3.connect("../data/movies_database.db")
df_movies.to_sql('df_movies_sql', conn, index=False)
df_credits.to_sql('df_credits_sql', conn, index=False)

# Define the SQL query for the join
query = """
    SELECT * 
    FROM df_movies_sql
    INNER JOIN df_credits_sql ON df_movies_sql.title = df_credits_sql.title
"""

# Execute the SQL query
df_total = pd.read_sql_query(query, conn)

# Close the SQLite connection
conn.close()

print("Result of the SQL-like join:")
print(df_total)


ValueError: Table 'df_movies_sql' already exists.

In [ ]:
df_total=df_total[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
df_total = df_total.loc[:, ~df_total.columns.duplicated()]
df_total

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
def load_json_safe(json_str, default_value = None):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return default_value


df_total["genres"] = df_total["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
df_total["keywords"] = df_total["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

df_total["cast"] = df_total["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)
df_total["crew"] = df_total["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in load_json_safe(x) if crew_member['job'] == 'Director']))
df_total["overview"] = df_total["overview"].apply(lambda x: [x])
df_total

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,[El Mariachi just wants to play his guitar and...,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,72766,Newlyweds,[A newlywed couple's honeymoon is upended by t...,"[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,231617,"Signed, Sealed, Delivered","[""Signed, Sealed, Delivered"" introduces a dedi...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,126186,Shanghai Calling,[When ambitious New York attorney Sam is sent ...,[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


In [ ]:
df_total.sample(10)

,movie_id,title,overview,genres,keywords,cast,crew
4721,86812,Sound of My Voice,[A journalist and his girlfriend get pulled in...,"[Science Fiction, Drama, Mystery]","[journalist, independent film, hand clapping g...","[Brit Marling, Christopher Denham, Nicole Vicius]",Zal Batmanglij
2122,21208,Orphan,[A married couple with a rocky past adopt 9-ye...,"[Horror, Thriller, Mystery]","[nun, deaf-mute, orphan, all girl, troubled ma...","[Vera Farmiga, Peter Sarsgaard, Isabelle Fuhrman]",Jaume Collet-Serra
4522,30315,Kingdom of the Spiders,[Investigating the mysterious deaths of a numb...,"[Horror, Science Fiction]","[hotel, small town, siege, independent film, c...","[William Shatner, Tiffany Bolling, Woody Strode]",John 'Bud' Cardos
1208,10761,Made of Honor,[Tom and Hannah have been platonic friends for...,"[Comedy, Romance]","[scotland, love of one's life, ladykiller, one...","[Patrick Dempsey, Michelle Monaghan, Kevin McK...",Paul Weiland
1872,9655,She's the Man,[Viola Johnson is in a real jam. Complications...,"[Comedy, Drama, Family, Romance]","[roommate, twin sister, sport, boarding school...","[Amanda Bynes, Channing Tatum, Laura Ramsey]",Andy Fickman
4043,13569,Thr3e,[Innocent lives hang on the whim of an elusive...,"[Drama, Horror, Thriller]",[],"[Marc Blucas, Justine Waddell, Bill Moseley]",Robby Henson
3201,17926,Undiscovered,[A group of aspiring entertainers try to estab...,"[Comedy, Romance]","[independent film, aspiring singer, singer, lo...","[Kip Pardue, Carrie Fisher, Shannyn Sossamon]",Meiert Avis
2447,79694,The Apparition,[Plagued by frightening occurrences in their h...,"[Horror, Thriller]","[experiment, supernatural, paranormal, hauntin...","[Ashley Greene, Sebastian Stan, Tom Felton]",Todd Lincoln
2729,801,"Good Morning, Vietnam",[Radio funny man Adrian Cronauer is sent to Vi...,"[Comedy, Drama, War]","[rock and roll, radio station, war crimes, ent...","[Robin Williams, Forest Whitaker, Tung Thanh T...",Barry Levinson
1343,2163,Breakdown,[When his SUV breaks down on a remote Southwes...,"[Drama, Action, Thriller]","[california, bank, ransom, car breakdown, kidn...","[Kurt Russell, J. T. Walsh, Kathleen Quinlan]",Jonathan Mostow


In [ ]:
df_tags = pd.DataFrame()
df_total["overview"] = df_total["overview"].apply(lambda x: [str(x)])
df_total["genres"] = df_total["genres"].apply(lambda x: [str(genre) for genre in x])
df_total["keywords"] = df_total["keywords"].apply(lambda x: [str(keyword) for keyword in x])
df_total["cast"] = df_total["cast"].apply(lambda x: [str(actor) for actor in x])
df_total["crew"] = df_total["crew"].apply(lambda x: [str(x)])


df_total["tags"] = df_total["overview"] + df_total["genres"] + df_total["keywords"] + df_total["cast"] + df_total["crew"]
df_total["tags"] = df_total["tags"].apply(lambda x: ",".join(x).replace(",", " "))

df_total.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)

df_total.iloc[0].tags


"['In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization.'] Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver James Cameron"

In [24]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_total["tags"])

model = NearestNeighbors(n_neighbors = 6, algorithm = "brute", metric = "cosine")
model.fit(tfidf_matrix)

def get_movie_recommendations(movie_title):
    movie_index = df_total[df_total["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df_total["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "George and the Dragon"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

Film recommendations 'George and the Dragon'
- Film: How to Train Your Dragon 2
- Film: Dragon Nest: Warriors' Dawn
- Film: Pete's Dragon
- Film: The Hobbit: The Desolation of Smaug
- Film: Dragonslayer
